In [1]:
# !pip install tensorflow==2.0.0-alpha0
# !pip install dfply

    100% |████████████████████████████████| 79.9MB 440kB/s 
    100% |████████████████████████████████| 61kB 20.4MB/s 
    100% |████████████████████████████████| 3.0MB 9.3MB/s 
    100% |████████████████████████████████| 419kB 12.6MB/s 
  Found existing installation: tensorflow 1.13.1
    Uninstalling tensorflow-1.13.1:
      Successfully uninstalled tensorflow-1.13.1
    100% |████████████████████████████████| 614kB 22.2MB/s 


In [0]:
import tensorflow as tf
import dfply as dp
import pandas as pd
import numpy as np
from toolz import pipe
import sklearn.model_selection as mselect
import sklearn.metrics as skmets
from google.colab import drive

In [3]:
drive.mount('/content/drive', force_remount=True)
# plt.imshow(preprocess_image(train_images[preview_index]), cmap="gray")


Mounted at /content/drive


In [0]:
parent = '/content/drive/My Drive/Colab Notebooks/'
df_train = pd.read_csv(parent + 'fashion-mnist_train.csv')
df_test = pd.read_csv(parent + 'fashion-mnist_test.csv')

In [0]:
# def standardize(*tensors):
#   return [t / 255 for t in tensors]

# def reshape(*tensors):
#   return [t.reshape(-1, 784) for t in tensors]
#
# def prep(*tensors):
#   return [pipe(t, standardize, reshape) for t in tensors]

def standardize(t):
  return t / 255

def reshape(t):
#   return t.reshape(-1, 784)
  return np.expand_dims(t, axis=-1)

def prep(t):
  return pipe(t,
              standardize,
              reshape)
  

def form_xy(df):
  # X = all the columns that do not have the name 'label'
  # y = the column that is named 'label'
  # return X, y
  return (df >> dp.select(~dp.X.label)).values,\
           (df >> dp.select(dp.X.label)).values


def nested_cv(df):
  X, y = form_xy(df)
  X = prep(X)
  rskf = mselect.RepeatedStratifiedKFold(n_splits=5, random_state=42)
  for train_i, test_i in rskf.split(X,y):
    pass

In [6]:
X,y = form_xy(df_train)
X = prep(X)
X.shape


(60000, 784, 1)

In [7]:
X[0].shape


(784, 1)

In [9]:
# shape = (X.shape[0],1,*X.shape)
# shape = (1,*X.shape)
# shape = X.shape
kernel = (3,3)
classes = 10
start = 8
end = 2 * start
layers = [
    tf.keras.layers.Conv2D(filters=start, kernel_size=kernel, activation=tf.nn.relu, input_shape=X[0].shape, data_format='channels_last'),
    tf.keras.layers.Conv2D(filters=end, kernel_size=kernel, activation=tf.nn.relu),
    tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=kernel),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=(2 * end), activation=tf.nn.relu),
    tf.keras.layers.Dense(units=start, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=classes, activation=tf.nn.softmax)
]

nnet = tf.keras.Sequential(layers)
nnet.compile(optimizer=tf.optimizers.Adam(),
             loss=tf.losses.sparse_categorical_crossentropy,
             metric=tf.metrics.sparse_categorical_accuracy)

ValueError: ignored

In [0]:
epochs = 5
batch = 50
nnet.fit(X, y, epochs=epochs, batch_size = batch)
nnet.save_weights['nnet.tf']

ValueError: ignored

In [0]:
?tf.keras.layers.Conv2D